In [1]:
# notebook last tested with abtem version 1.0.0beta32
%matplotlib inline

from abtem import __version__
print('current version:', __version__)

from ase.io import read
from ase.cluster import wulff_construction
from ase import Atoms
import ase.data
import matplotlib.pyplot as plt
from scipy import constants
from abtem import *
import numpy as np

/zhome/ab/7/153983/project/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


current version: 1.0.0beta32


In [2]:
rotations = [[127.6, 51.5, 231.8],
             [140.1, 50.4, 225  ],
             [140.4, 49.8, 224.8],
             [243,   52,   127.6]]

In [3]:
center_type = 'com' 

In [4]:
for rotation in rotations: 
    
    #Creating atoms
    surfaces = [(1, 0, 0), (1, 1, 0), (1, 1, 1)] # Nanoparticle facets
    esurf = [1.0, 1.1, 0.9] # Relative surface energies
    size = 16000 # Target number of atoms
    atoms = Atoms(wulff_construction('Au', surfaces, esurf, size, 'fcc', rounding='above'))
    atoms.rotate(rotation[0], 'z', center='cop')
    atoms.rotate(rotation[1], 'y', center='cop')
    atoms.rotate(rotation[2], 'z', center='cop')
    atoms.center(vacuum=5.)
    atoms.center(vacuum=2., axis=2)
    
    # Setting gridpoint config
    gridpoints = int(max(atoms.cell[0,0], atoms.cell[1,1]) / 0.05)
    gridpoints = 4096
    
    #Generating images
    angle = 2*4.5e-3
    defocus = 40/np.sin(angle)
    mass = ase.data.atomic_masses[ase.data.atomic_numbers['Au']]
    TD = 165  # Kittel
    T = 300
    U_squared = (3 * constants.hbar**2 / (2 *  mass * ase.units._amu * constants.k * TD)
                 * (1/4 + T / TD))
    sigmas = {'Au': np.sqrt(U_squared) * 1e10} # standard deviations of thermal vibrations
    print(sigmas)
    num_configs = 10 # number of frozen phonon configurations

    frozen_phonons = FrozenPhonons(atoms, num_configs=num_configs, sigmas=sigmas)

    for i,config in enumerate(frozen_phonons):
        potential = Potential(config, 
                              gpts=gridpoints, 
                              slice_thickness=0.5, 
                              device='gpu',
                              #storage='cpu',
                              precalculate=False,
                              parametrization='kirkland', 
                              projection='infinite') # ASK

        probe = Probe(sampling=potential.sampling,
            device='gpu',
            semiangle_cutoff=1000*angle,
            extent=potential.extent,
            defocus=defocus,
            energy=30e3 # acceleration voltage in eVhttp://localhost:8080/notebooks/Kikuchi.ipynb#
        )
        positions = (45, 45)
        %time exit_wave = probe.multislice(positions, potential)
        cbed_diffraction_pattern = exit_wave.diffraction_pattern(block_zeroth_order=0)
        if (i==0):
            summedImage = cbed_diffraction_pattern.array
        else:
            summedImage += cbed_diffraction_pattern.array

    cbed_diffraction_pattern.array[:] = summedImage/num_configs
    exit_wave.array.shape

    # Deleting probe
    energy = probe.energy
    del probe
    
    # Switching back to CPU
    if True:
        exit_wave._array = exit_wave._array.get()
        exit_wave._device = 'cpu'

    plt.savefig('image-{}.png'.format("{}-{}-{}".format(rotation[0], rotation[1], rotation[2])))

{'Au': 0.0680474303682167}


Multislice: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 160/160 [00:05<00:00, 27.22it/s]


CPU times: user 3.18 s, sys: 3.23 s, total: 6.41 s
Wall time: 6.47 s


Multislice: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 160/160 [00:05<00:00, 31.30it/s]


CPU times: user 2.52 s, sys: 2.63 s, total: 5.14 s
Wall time: 5.12 s


...

Multislice: 100%|██████████| 171/171 [00:06<00:00, 27.17it/s]


CPU times: user 3.09 s, sys: 3.25 s, total: 6.34 s
Wall time: 6.32 s


Multislice: 100%|██████████| 171/171 [00:06<00:00, 27.05it/s]


CPU times: user 3.12 s, sys: 3.25 s, total: 6.36 s
Wall time: 6.34 s


<Figure size 640x480 with 0 Axes>